In [33]:
import csv
import chars2vec
import re
import numpy as np
import keras.backend as K
from tensorflow.keras import datasets, layers, models, losses, callbacks
from utils import read_text_data, f1
from sklearn.model_selection import train_test_split
import statistics

Define a new import function as preprocessing isn't in this notebook

In [37]:
def read_data_span(filename):
    """Reads csv file with python, text."""
    data = []
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile)
        count = 0
        for row in reader:
            if row['span'] == '[]' or row['span'] == []:
                data.append([])
            else:
                data.append([int(j) for j in row['span'][1:-1].split(", ")])
    csvfile.close()
    return data

Read the test text and labels

In [38]:
texts = read_text_data('data/tsd_test_readable.csv')
spans = read_data_span('data/tsd_test_readable.csv')

In [39]:
##ADDED FOR PRESENTATION##

def add_to_dict(filename):
    with open(filename) as read_list:
    
        reader = csv.reader(read_list)
        for row in reader:
            toxic_dictionary[row[0]] = 1
    read_list.close()

Inspect the text

In [40]:
texts

["That's right. They are not normal. And I am starting from the premise that they are ABNORMAL. Proceed wth the typical racist, bigot, sexist rubbish.\nThanks!",
 '"Watch people die from taking away their healthcare"\n\nDING DING DING!  Winner of stupid post of the day award!',
 'tens years ago i contacted the PDR and suggested that the time might be good to work with alaska on building a gas line.. alaska rejected them without even consideration despite china being flush with cash and hungry for gas.. and set up another infamous boondoggle.. the transcanada-exxon rip off that we are still paying for and have yet to receive anything of value.. hundreds of millions of dollars on studies.. and buyouts.. i hope china says f you alaska.. you are nothing but ignorant people..',
 'The parallels between the ANC and the Sicilian Mafia are glaring. The ANC has always been run by a few "families" who treat the state as \'turf\' ; as just one big piggy bank for their self-enrichment. The governme

Inspect the spans

In [41]:
spans

[[84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146],
 [81, 82, 83, 84, 85, 86],
 [],
 [],
 [],
 [129, 130, 131, 132, 133, 134],
 [35, 36, 37, 38, 39, 40, 41, 42, 43],
 [55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129],
 [37, 38, 39, 40, 41, 158, 159, 160, 161, 204, 205, 206, 207, 208],
 [94, 95, 96, 97],
 [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64],
 [201, 202, 203, 204, 205, 206, 207, 208, 209, 210],
 [30,
  31,
  32,
  33,
  34,
  35,
  36,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  52,
  53,
  54,
  55,
  56,
  57,
  58],
 [134, 135, 136, 137, 138, 139],
 [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
 [],
 [2, 3, 4, 5],
 [],
 [],
 [0, 1, 2, 3, 4],
 [12, 13, 14, 

Load the test data into a numpy array

In [42]:
test_X = np.zeros(shape=(len(texts), 1024, 50))
c2v_model = chars2vec.load_model('eng_50')
for x, string in enumerate(texts):
    for y, char in enumerate(string):
            char_vect = c2v_model.vectorize_words([char])
            test_X[x][y] = [word_vect for word_vect in char_vect[0]]

In [43]:
example = np.zeros(shape=(1, 1024, 50))
test_string = ["You wanker, you fucking egit I hope you die you a**hole"]

for x, string in enumerate(test_string):
    for y, char in enumerate(string):
            char_vect = c2v_model.vectorize_words([char])
            example[x][y] = [word_vect for word_vect in char_vect[0]]

In [44]:
model = models.load_model(f"DeconvNet_model_300_epochs")
y_pred = model.predict(test_X)

In [45]:
def fix_word_boundaries(span, text):
    # "You fucking Moron you silly cunt" [6,7,8,9,10,11,12,13,14,15,16,28,29,30]
    # [4,5,6,7,8,9,10,11,12,13,14,15,16,28,29,30,31]
    seperated_text = []
    word = ''
    new_span = []
    current_word_span = []
    toxic_word = False
    for n, char in enumerate(text):
        if n in span:
            toxic_word = True
        if char == ' ':
            seperated_text.append(word)
            seperated_text.append(' ')
            word = ''
            if toxic_word:
                new_span.extend(current_word_span)
                current_word_span = []
                toxic_word = False
            else:
                current_word_span = []
                toxic_word = False
        else:
            word += char
            current_word_span.append(n)
        if n == len(text) - 1:
            seperated_text.append(word)
            if toxic_word:
                new_span.extend(current_word_span)
    return new_span

In [46]:
scores = []
for x, pred in enumerate(y_pred):
    y_pred_f1_compatible = [j for j, i in enumerate(pred) if np.argmax(i) == 0]
    #y_pred_f1_compatible = fix_word_boundaries(y_pred_f1_compatible, texts[x])
    y_true_f1_compatible = spans[x]
    score = f1(y_pred_f1_compatible, y_true_f1_compatible)
    scores.append(score)

print('avg F1 %g' % statistics.mean(scores))


avg F1 0.58718


In [47]:
test_pred = model.predict(example)

In [48]:
for x, pred in enumerate(test_pred):
    char_arr = [j for j, i in enumerate(test_pred[0]) if np.argmax(i) == 0]
    print(f"text: {test_string[x]}")
    print(f"Predicted span: {char_arr}")
    print(f"Flagged text: {test_string[x][char_arr[0]:char_arr[-1]]}")
    

text: You wanker, you fucking egit I hope you die you a**hole
Predicted span: [16, 17, 18, 19, 20, 21, 22]
Flagged text: fuckin


In [49]:
##ADDED FOR PRESENTATION##

import csv
import pandas as pd

with open("data/tsd_train_readable.csv", encoding="utf-8", errors="ignore") as csv_file:
    with open("data/tsd_train_ground_truth_words.csv", "w", newline="") as out_file:
        reader = csv.reader(csv_file)
        writer = csv.writer(out_file)
        for row in reader:
            
            invalid_span = False
            complete_span = False
            string_vector = ""
            phrases = []
            if row[0] == "[]":
                offset_vector = []
                string_vector = row[1]
                complete_span = True
            else:
                offset_vector = row[0][1:-1].split(", ")
                offset_vector_int = []
                for item in offset_vector:
                    if item.isnumeric():
                        offset_vector_int.append(int(item))
                    else:
                        invalid_span = True
                        break
                offset_vector = offset_vector_int
                for string_vector_index, char_index in enumerate(offset_vector):
                    if invalid_span:
                        break
                    if complete_span:
                        break
                    if char_index > len(row[1])-1:
                        break
                    if string_vector_index == 0:
                        string_vector = string_vector + row[1][int(char_index)]
                    else:
                        if int(char_index) != offset_vector[string_vector_index-1] + 1:
                            if len(string_vector.split(" ")) < 2:
                                phrases.append(string_vector)
                                string_vector = ""
                                string_vector = string_vector + row[1][int(char_index)]
                        else:
                            string_vector = string_vector + row[1][int(char_index)]
                if len(string_vector.split(" ")) < 2:
                    phrases.append(string_vector)
                if complete_span == False and string_vector != "" and phrases != []:
                    for phrase in phrases:
                        print(phrase)
                        writer.writerow([phrase])
csv_file.close()
out_file.close()
                        
with open("data/tsd_trial_readable.csv", encoding="utf-8", errors="ignore") as csv_file:
    with open("data/tsd_train_ground_truth_words.csv", "a", newline="") as out_file:
        reader = csv.reader(csv_file)
        writer = csv.writer(out_file)
        for row in reader:
            
            invalid_span = False
            complete_span = False
            string_vector = ""
            phrases = []
            if row[0] == "[]":
                offset_vector = []
                string_vector = row[1]
                complete_span = True
            else:
                offset_vector = row[0][1:-1].split(", ")
                offset_vector_int = []
                for item in offset_vector:
                    if item.isnumeric():
                        offset_vector_int.append(int(item))
                    else:
                        invalid_span = True
                        break
                offset_vector = offset_vector_int
                for string_vector_index, char_index in enumerate(offset_vector):
                    if invalid_span:
                        break
                    if complete_span:
                        break
                    if char_index > len(row[1])-1:
                        break
                    if string_vector_index == 0:
                        string_vector = string_vector + row[1][int(char_index)]
                    else:
                        if int(char_index) != offset_vector[string_vector_index-1] + 1:
                            if len(string_vector.split(" ")) < 2:
                                phrases.append(string_vector)
                                string_vector = ""
                                string_vector = string_vector + row[1][int(char_index)]
                        else:
                            string_vector = string_vector + row[1][int(char_index)]
                if len(string_vector.split(" ")) < 2:
                    phrases.append(string_vector)
                if complete_span == False and string_vector != "" and phrases != []:
                    for phrase in phrases:
                        print(phrase)
                        writer.writerow([phrase])
csv_file.close()
out_file.close()

fucking
Damn
knucklehead
killing
stupid
emoticons
Dumb
idiots
stupid
pathetic
imbecile
idiot
repugnant
joke
protest
stupid
sociopathic
parasitic
sycophants
bum
dumb
Kill
fool
ignorant
Damn
stupid
mistake
scam
ignorant
lunatics
ignorant
testicles
stupid
moron
Bury
Morons
scum
incompetent
bunglers
idiots
background
stupid
idiots
morons
stupid
hypocrite
ignorant
hypocrisy
Darn
coward
coward
loser
died
shit
idiotic
idiots
crazed
crap
stupid
stupid
demeans
nuts
challenging
These
Immoral
idiot
stupid
jerks
loser
MORONS
idiots
moron
steals
scumbag
corrupt
idiotic
crap
outrage
Folks
ignorant
Fool
idiots
idiot
useless
fool
Stupid
cuck
stupidest
idiot
idiots
ducks
stupid
stupid
semen
gay
imbeciles
fool
ANTIFA
BLM
silly
stupifying
idiot
dumb
imbecilic
trash
Thief
idiots
asses
idiots
idiot
morons
obnoxious
stupidity
Idiot
drugs
idiots
idiotic
idiot
Russians
stupid
stupid
stupid
hypocrite
sick
puppy
animal
idiots
Stupid
stupidity
Stupidity
buffoon
stupidity
kill
bullet
would
stupid
jerk
government


imbecile
stupid
gross
disgusting
vile
stupid
shit
shit
stupidity
bullshit
stupidity
fool
morons
suck
stupid
Anyone
stupid
pigs
lipstick
morons
idiots
idiots
stupidity
pig
idiotic
loser
suck
idiocy
idiotic
stupid
hypocrisy
idiot
scum
Useless
vagrancy
politicians
pathetic
stupid
stupidity
ignorance
idiot
ignorant
stupid
Idiot
stupid
always
nasty
rude
stupid.It's
stupidest
fool
weewili
stupid
stupid
stupid
such
seals
jerk
idiotic
idiots
stupidity
entitled
stupid
bitch
idiot
total
ridiculous
suckers
sex
stupid
Bullshit
Suck
ignorant
idiot
stupidity
dumb
Idiots
scum
mentor
because
dumb
stupid
stupid
Silly
dick
suckers
suckers
moron
Donald
idiot
idiot
fool
Judass
beast
idiots
shit
FUCKING
moron
idiotic
stupid
bitch
moron
stupidity
Tillerson
moron
Nazis
stupid
murderer
dummy
IDIOT
clowns
Stupid
pathetic
arrogance
hypocrisy
Golden
stupid
moron
stupid
idiot
fool
crook
morons
shoot
stupid
dumb
know
imbecile
moron
stupid
killer
ludicrous
fool
worthless
stupid
Stupid
fool
stupid
robbed
stupid
nons

stupid
PIG
S__t
Idiots
damned
art
ignorant
stupid
stupid
garbage
left's
stupidity
evangelicals
Catholics
stupid
stupid
idiots
idiot
stupidity
stupidity
Kill
ignorant
arse
ignorant
fool
stupid
stupid
stupid
dumb
stupid
Stupid
Idiot
Traitor
moron
scum
trolls
idiot
stupid
repulsive
troll
clown
cretin
Fools
hypocrites
stupidity
stupid
Damn
fool
stupid
class
crap
rape
stupid
stupidity
idiots
hypocrites
trolls
Stupid
idiot
things
stupid
troll
stupid
stupidity
pathetic
Liberals
stupid
masturbation
could
dumb
killing
animals
stupid
crazy
pig
stupid
morons
ignorant
sick
scumbag
b*itch
Damn
pathetic
idiots
stupidity
stupid
stupid
idiots
dumbest
idiots
coward
sucks
foolish
Stupid
stupid
idiotic
engineers
racist
trudeau
stupid
pathetic
garbage
Idiot
These
sucks
hypocrites
Terrorizing
Idiot
sociopath
POTUS
crazy
crap
fool
idiot
stupid
idiot
stupid
ridicule
crook
scum
idiot
idiots
kill
idiot
jerk
stupidity
idiot
Stupidity
Gross
Scum
shit
Cowards
bullshit
fool
pathetic
idiot
dumb
loser
IDIOT
STUPID
c

ignorant
dumb
crap
saying
stupid
stupid
idiot
stupid
stupid
stupid
indian
moron
homophobic
idiot
silly
stupida
ridiculous
clown
Stupid
cowards
hypocrites
idiots
stupidity
animals
trash
moron
stupid
lying
corrupt
crook
stupid
crap
gonna
shit
liars
ignorant
idiots
stupid
stupid
ignorance
blather
ignorant
idiots
stupid
stupidity
stupidity
stupid
stupid
stupidity
stupidity
stupidity
stupid
stupid
stupid
stupid
clowns
Hell
scumbag
homosexual
I*D*I*O*T*S
crappy
jerk
Idiots
idiot
stupid
immoral
stupid
stupid
ugly
STUPID
damn
fool
joined
dumb
loser
loser
loser
losers
traitor
darn
disease
stupid
idiots
killed
criminals
damn
stupidity
ignorant
clowns
IDIOT
moron
ridiculous
coward
BULLY
like
anonymous
morons
righteous
Pathetic
horrible
idiot
dumb
Pathetic
Kill
idiot
racis
sexist
fool
moron
idiotic
stupid
dick
crap
jerk
ignorant
trash
garbage
kill
idiot
pathetic
idiocy
idiot
dumb
Dumb
crap
incompetent
coward
But
moron
F
U
C
K
stupid
bastards
pussy
shows
idiot
folks
Weintraub
pathetic
dickhead
clow

In [50]:
##ADDED FOR PRESENTATION##
toxic_dictionary = dict({})
add_to_dict("data/tsd_train_ground_truth_words.csv")

test = read_text_data("data/tsd_test_readable.csv")

In [56]:
##ADDED FOR PRESENTATION##
scores = []
for x, text in enumerate(test):
    predict_spans = []
    for phrase in toxic_dictionary.keys():
        if text.find(phrase) != -1:
            predict_spans.extend(range(text.find(phrase), text.find(phrase) + len(phrase)))
            break
    predict_spans = list(set(predict_spans))
    ensemble = set(predict_spans).intersection([j for j, i in enumerate(y_pred[x]) if np.argmax(i) == 0])
    score = f1(ensemble, spans[x])
    scores.append(score)

print('avg F1 %g' % statistics.mean(scores))

avg F1 0.564603
